## Project Overview
In this project, we aim to provide insights for Microsoft's new movie studio by analyzing movie data from Box Office Mojo and TMDB. The business problem is to understand which types of films are currently successful at the box office. We will use the bom.movie_gross.csv and tmdb.movies.csv datasets to explore film genres, ratings, and earnings. Our analysis will involve data cleaning, exploration, and visualization using pandas and seaborn. Based on our findings, we will offer recommendations to help Microsoft decide what types of films to produce.

## Business Problem:
Microsoft wants to enter the movie industry but lacks insights into what types of films are currently successful. The key questions we aim to answer are:

What genres are performing best at the box office?
What is the relationship between movie ratings and earnings?
How do box office earnings vary across different genres?
These questions are crucial for Microsoft to make informed decisions about the types of films to create, ensuring they invest in genres that attract large audiences and generate high revenue.

### Data Understanding:
The data for this project comes from Box Office Mojo and IMDB.

In [ ]:
# import packages
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt

In [1]:
# load data
df1 = pd.read_csv('Data/bom.movie_gross.csv')
df2 = pd.read_csv('Data/tmdb.movies.csv')

print(df1.head())
print(df2.head())

NameError: name 'pd' is not defined